In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [3]:
dataold = pd.read_csv(r"./traj.csv",index_col = 0)

In [4]:
# 分解坐标字段
def split_coordinates(coord_str):
    coord = eval(coord_str)
    return coord[0], coord[1]
# 解析字符串为 datetime
dataold['timestamp_column'] = pd.to_datetime(dataold['time'])

# 转换为 Unix 时间戳（单位为秒）
dataold['unix_timestamp'] = dataold['timestamp_column'].astype('int64') // 10**9
#print(data)
# 分解坐标
dataold[['longitude', 'latitude']] = dataold['coordinates'].apply(lambda x: pd.Series(split_coordinates(x)))

#X = data[['entity_id', 'longitude', 'latitude', 'current_dis', 'speeds','holidays']]  # 特征
#y = data[['latitude', 'longitude']]  # 预测目标，例如经纬度


In [5]:
data = dataold.drop(['coordinates', 'timestamp_column', 'time'], axis=1)

In [6]:
# 标准化数据
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[['longitude', 'latitude']])

In [7]:
# 使用 K-means 聚类
kmeans = KMeans(n_clusters=100)  # 假设我们划分为10个区域
data['cluster'] = kmeans.fit_predict(data_scaled)

In [8]:
# 同时对经度和纬度列下移一行
data['prev_longitude'] = data['longitude'].shift(1)
data['prev_latitude'] = data['latitude'].shift(1)
data['prev_dis'] = data['current_dis'].shift(1)
data['prev_traj_id'] = data['traj_id'].shift(1)

data.dropna(inplace=True)  # 移除由于shift造成的缺失值行
data


,entity_id,traj_id,current_dis,speeds,holidays,unix_timestamp,longitude,latitude,cluster,prev_longitude,prev_latitude,prev_dis,prev_traj_id
id,,,,,,,,,,,,,
1,254,0,0.562623,24.5375,0,1381254405,116.315102,40.004784,52,116.318726,40.009014,0.000000,0.0
2,254,0,0.778695,31.9675,0,1381254459,116.315018,40.002842,36,116.315102,40.004784,0.562623,0.0
3,254,0,1.252148,19.7850,0,1381254566,116.315041,39.998585,36,116.315018,40.002842,0.778695,0.0
4,254,0,1.924533,24.4500,0,1381254675,116.315605,39.992554,73,116.315041,39.998585,1.252148,0.0
5,254,0,2.448189,25.1500,0,1381254729,116.315735,39.987846,73,116.315605,39.992554,1.924533,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518403,4954,21998,21.419211,27.1450,0,1383174875,116.333366,39.790730,28,116.335815,39.790680,21.209873,21998.0
518404,4954,21998,21.669183,30.0825,0,1383174899,116.330444,39.790840,28,116.333366,39.790730,21.419211,21998.0
518405,4954,21998,22.129037,46.0550,0,1383174995,116.328453,39.793888,28,116.330444,39.790840,21.669183,21998.0


In [9]:
data = data[data['prev_traj_id'] == data['traj_id']]
data

,entity_id,traj_id,current_dis,speeds,holidays,unix_timestamp,longitude,latitude,cluster,prev_longitude,prev_latitude,prev_dis,prev_traj_id
id,,,,,,,,,,,,,
1,254,0,0.562623,24.5375,0,1381254405,116.315102,40.004784,52,116.318726,40.009014,0.000000,0.0
2,254,0,0.778695,31.9675,0,1381254459,116.315018,40.002842,36,116.315102,40.004784,0.562623,0.0
3,254,0,1.252148,19.7850,0,1381254566,116.315041,39.998585,36,116.315018,40.002842,0.778695,0.0
4,254,0,1.924533,24.4500,0,1381254675,116.315605,39.992554,73,116.315041,39.998585,1.252148,0.0
5,254,0,2.448189,25.1500,0,1381254729,116.315735,39.987846,73,116.315605,39.992554,1.924533,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518403,4954,21998,21.419211,27.1450,0,1383174875,116.333366,39.790730,28,116.335815,39.790680,21.209873,21998.0
518404,4954,21998,21.669183,30.0825,0,1383174899,116.330444,39.790840,28,116.333366,39.790730,21.419211,21998.0
518405,4954,21998,22.129037,46.0550,0,1383174995,116.328453,39.793888,28,116.330444,39.790840,21.669183,21998.0


In [10]:

# 特征
X = data[['entity_id','traj_id', 'prev_longitude', 'prev_latitude','prev_dis', 'speeds','holidays','unix_timestamp']]
# 标签
y = data['cluster']

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.ensemble import RandomForestClassifier
# 创建随机森林模型
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [13]:
from sklearn.metrics import classification_report

# 在测试集上进行预测
y_pred = rf.predict(X_test)

# 评估模型
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.88      0.85      0.87      1350
           1       0.86      0.85      0.86       976
           2       0.87      0.89      0.88      1137
           3       0.87      0.87      0.87      1435
           4       0.84      0.88      0.86      1078
           5       0.86      0.84      0.85       423
           6       0.85      0.87      0.86       519
           7       0.88      0.88      0.88      1084
           8       0.82      0.81      0.82       677
           9       0.93      0.92      0.93       558
          10       0.95      0.92      0.94       508
          11       0.86      0.83      0.85       765
          12       0.84      0.86      0.85      1348
          13       0.86      0.88      0.87       970
          14       0.94      0.91      0.93       257
          15       0.94      0.93      0.93       464
          16       0.90      0.92      0.91       438
          17       0.92    

In [14]:
# 对于分类模型
accuracy = rf.score(X_test, y_test)
accuracy

0.8668831577489962

In [15]:
data.head()
X1 = data[['entity_id','traj_id', 'prev_longitude', 'prev_latitude', 'prev_dis','speeds','holidays','unix_timestamp','cluster']]
# 特征 drop 方法用于从 data DataFrame 中移除特定的列。
y1 = data[['longitude','latitude']]  

In [16]:
# 划分数据集
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)


In [17]:
# 创建随机森林回归模型
from sklearn.ensemble import RandomForestRegressor

model1 = RandomForestRegressor(n_estimators=100, random_state=42)
model1.fit(X1_train, y1_train)

RandomForestRegressor(random_state=42)

In [18]:
# 在测试集上进行预测
predictions = model1.predict(X1_test)

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
# predictions 是模型的预测结果，y_test 是实际的标签
mse = mean_squared_error(y1_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y1_test, predictions)

print(f"均方误差 (MSE): {mse}")
print(f"均方根误差 (RMSE): {rmse}")
print(f"平均绝对误差 (MAE): {mae}")

均方误差 (MSE): 4.546350540798811e-06
均方根误差 (RMSE): 0.0021322172827361687
平均绝对误差 (MAE): 0.0012938083315415093


In [20]:
r2 = model1.score(X1_test, y1_test)
r2

0.9988354442915142

In [21]:
import pandas as pd
import numpy as np
# 加载数据
missing_data = pd.read_csv('./data_dis.csv')



In [22]:
# 同时对经度和纬度列下移一行
missing_data['prev_coordinates'] = missing_data['coordinates'].shift(1)
missing_data['prev_dis'] = missing_data['current_dis'].shift(1)
missing_data['prev_traj_id'] = missing_data['traj_id'].shift(1)
missing_data


,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,prev_coordinates,prev_dis,prev_traj_id
0,355,2013-10-08T08:30:00Z,256,25,"[116.324127,39.897049]",0.000000,21.6075,0,NaN,NaN,NaN
1,356,2013-10-08T08:30:55Z,256,25,"[116.327652,39.897018]",0.300751,21.1500,0,"[116.324127,39.897049]",0.000000,25.0
2,357,2013-10-08T08:32:44Z,256,25,"[116.330978,39.897041]",0.584521,20.4825,0,"[116.327652,39.897018]",0.300751,25.0
3,358,2013-10-08T08:34:32Z,256,25,"[116.336624,39.897305]",1.067123,20.6575,0,"[116.330978,39.897041]",0.584521,25.0
4,359,2013-10-08T08:35:25Z,256,25,"[116.341118,39.897537]",1.451388,24.0700,0,"[116.336624,39.897305]",1.067123,25.0
...,...,...,...,...,...,...,...,...,...,...,...
29380,518325,2013-10-30T22:09:12Z,4954,21997,"[116.328094,39.967125]",3.765071,39.0475,0,"[116.32357,39.966927]",3.378877,21997.0
29381,518326,2013-10-30T22:09:43Z,4954,21997,"[116.330856,39.967224]",4.000737,53.7950,0,"[116.328094,39.967125]",3.765071,21997.0
29382,518327,2013-10-30T22:11:26Z,4954,21997,"[116.333565,39.967266]",4.232179,44.0800,0,"[116.330856,39.967224]",4.000737,21997.0
29383,518328,2013-10-30T22:12:12Z,4954,21997,"[116.334488,39.963161]",4.695391,46.4450,0,"[116.333565,39.967266]",4.232179,21997.0


In [23]:
df=missing_data
# 找出 'target_column' 中包含空值的行
test_set = df[df['coordinates'].isna()]
test_set = test_set.copy()  # 创建train_set的副本
# 找出 'target_column' 中不包含空值的行
train_set = df[df['coordinates'].notna()]
train_set = train_set.copy()  # 创建train_set的副本
train_set.dropna(subset=['prev_coordinates'], inplace=True)  # 在副本上删除缺失值行

train_set


,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,prev_coordinates,prev_dis,prev_traj_id
1,356,2013-10-08T08:30:55Z,256,25,"[116.327652,39.897018]",0.300751,21.1500,0,"[116.324127,39.897049]",0.000000,25.0
2,357,2013-10-08T08:32:44Z,256,25,"[116.330978,39.897041]",0.584521,20.4825,0,"[116.327652,39.897018]",0.300751,25.0
3,358,2013-10-08T08:34:32Z,256,25,"[116.336624,39.897305]",1.067123,20.6575,0,"[116.330978,39.897041]",0.584521,25.0
4,359,2013-10-08T08:35:25Z,256,25,"[116.341118,39.897537]",1.451388,24.0700,0,"[116.336624,39.897305]",1.067123,25.0
5,360,2013-10-08T08:36:20Z,256,25,"[116.345924,39.89782]",1.862608,20.6550,0,"[116.341118,39.897537]",1.451388,25.0
...,...,...,...,...,...,...,...,...,...,...,...
29379,518324,2013-10-30T22:08:36Z,4954,21997,"[116.32357,39.966927]",3.378877,40.6125,0,"[116.322151,39.965206]",3.041784,21997.0
29380,518325,2013-10-30T22:09:12Z,4954,21997,"[116.328094,39.967125]",3.765071,39.0475,0,"[116.32357,39.966927]",3.378877,21997.0
29381,518326,2013-10-30T22:09:43Z,4954,21997,"[116.330856,39.967224]",4.000737,53.7950,0,"[116.328094,39.967125]",3.765071,21997.0
29382,518327,2013-10-30T22:11:26Z,4954,21997,"[116.333565,39.967266]",4.232179,44.0800,0,"[116.330856,39.967224]",4.000737,21997.0


In [24]:
import pandas as pd

# 添加经度和纬度列，并将它们的值设置为 0
test_set['longitude'] = 0
test_set['latitude'] = 0

In [25]:
# 解析字符串为 datetime
train_set['timestamp_column'] = pd.to_datetime(train_set['time'])
test_set['timestamp_column'] = pd.to_datetime(test_set['time'])

# 转换为 Unix 时间戳（单位为秒）
train_set['unix_timestamp'] = train_set['timestamp_column'].astype('int64') // 10**9
test_set['unix_timestamp'] = test_set['timestamp_column'].astype('int64') // 10**9

In [26]:
# 分解坐标
train_set[['longitude', 'latitude']] = train_set['coordinates'].apply(lambda x: pd.Series(split_coordinates(x)))

test_set[['prev_longitude', 'prev_latitude']] = test_set['prev_coordinates'].apply(lambda x: pd.Series(split_coordinates(x)))

In [27]:
# 使用模型进行预测
test_clusters = rf.predict(test_set[['entity_id','traj_id', 'prev_longitude', 'prev_latitude','prev_dis', 'speeds','holidays','unix_timestamp']])

test_set['cluster'] = test_clusters
test_set


,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,prev_coordinates,prev_dis,prev_traj_id,longitude,latitude,timestamp_column,unix_timestamp,prev_longitude,prev_latitude,cluster
14,369,2013-10-08T08:52:13Z,256,25,NaN,5.946249,22.6325,0,"[116.387314,39.900158]",5.422169,25.0,0,0,2013-10-08 08:52:13+00:00,1381222333,116.387314,39.900158,7
29,561,2013-10-08T08:40:24Z,257,40,NaN,4.435059,55.8975,0,"[116.430077,39.795883]",3.923330,40.0,0,0,2013-10-08 08:40:24+00:00,1381221624,116.430077,39.795883,35
44,1054,2013-10-08T18:38:44Z,257,53,NaN,4.728837,53.3450,0,"[116.339165,39.897507]",4.242190,53.0,0,0,2013-10-08 18:38:44+00:00,1381257524,116.339165,39.897507,99
59,1453,2013-10-08T06:12:17Z,4118,67,NaN,6.333737,69.3475,0,"[116.378769,39.968102]",5.827628,67.0,0,0,2013-10-08 06:12:17+00:00,1381212737,116.378769,39.968102,0
74,2077,2013-10-08T00:05:08Z,4123,105,NaN,5.336746,52.3975,0,"[116.443245,39.965462]",4.884481,105.0,0,0,2013-10-08 00:05:08+00:00,1381190708,116.443245,39.965462,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29324,517501,2013-10-30T11:41:56Z,4953,21962,NaN,5.281930,15.5675,0,"[116.295059,39.884083]",4.885816,21962.0,0,0,2013-10-30 11:41:56+00:00,1383133316,116.295059,39.884083,54
29339,517851,2013-10-30T08:11:50Z,4954,21980,NaN,4.471676,45.4225,0,"[116.38092,39.870617]",4.049021,21980.0,0,0,2013-10-30 08:11:50+00:00,1383120710,116.380920,39.870617,39
29354,517875,2013-10-30T08:47:15Z,4954,21981,NaN,5.508726,32.4525,0,"[116.443626,39.889412]",5.116139,21981.0,0,0,2013-10-30 08:47:15+00:00,1383122835,116.443626,39.889412,38
29369,517930,2013-10-30T09:57:59Z,4954,21983,NaN,5.070322,33.2825,0,"[116.392792,39.872326]",4.688592,21983.0,0,0,2013-10-30 09:57:59+00:00,1383127079,116.392792,39.872326,39


In [28]:
test_loc = model1.predict(test_set[['entity_id','traj_id', 'prev_longitude', 'prev_latitude', 'prev_dis','speeds','holidays','unix_timestamp','cluster']])

In [29]:
test_loc

array([[116.39018287,  39.90024991],
       [116.430434  ,  39.79580925],
       [116.33083947,  39.89722328],
       ...,
       [116.44410048,  39.88761256],
       [116.38804418,  39.87195588],
       [116.33509251,  39.96148785]])

In [30]:

for i, index in enumerate(test_set.index):
    missing_data.at[index, 'coordinates'] = test_loc[i]

In [31]:
# 将更改后的 DataFrame 保存为新文件
missing_data.to_csv('./data_all.csv', index=False)
